In [ ]:
#import all dependencies usind keras with tensowflow for backend
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.preprocessing import image
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten
import numpy as np

In [ ]:
#NOTE:  comment whole code block if you are not using Google colab to train the model
#-----------------------------------------------------------------------------
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1fMLi18fksmurRk5mFRKH7bp9aydqsKx0'
downloaded = drive.CreateFile({'id': file_id})

downloaded.GetContentFile('fer2013.csv')  




In [ ]:
#Did not use pandas library because dataset is already labled as training and testing. However we could used pandas too. But little lengty
with open("./fer2013.csv") as file:
    content_of_file = file.readlines()
 
content_array = np.array(content_of_file) 
numof_instances = content_array.size

In [ ]:
numof_instances

In [ ]:
numClasses = 7 #Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral
batchSize = 256
epochs = 5

In [ ]:
x_train, y_train, x_test, y_test = [], [], [], []

In [ ]:
for i in range(1,numof_instances):  
    emotion, img, usage = content_array[i].split(",")

    val = img.split(" ")
    pixels = np.array(val, 'float32')

    emotion = keras.utils.to_categorical(emotion, numClasses)

    if 'Training' in usage:
        y_train.append(emotion)
        x_train.append(pixels)
    elif 'PublicTest' in usage:
        y_test.append(emotion)
        x_test.append(pixels)


In [ ]:
x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')

In [ ]:
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')

In [ ]:
x_train /= 255
x_test /= 255

In [ ]:
x_test.shape

In [ ]:
x_train.shape

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_train = x_train.astype('float32')
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)
x_test = x_test.astype('float32')

In [ ]:
x_train.shape[0]
x_test.shape[0]

In [ ]:
#I got this structure from one of the blog of the link which I got from Youtube as I needed this to my PBL project and I needed highest accuracy
facial_model = Sequential()

# out put shape can be calculated by: i/p shape - filter_size + 1
#number of parameter which will be learned can be calculated for conv layer {filter_size * filter_size * channel(i/p) + bias}*total_filters
facial_model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48,48,1)))
facial_model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))

facial_model.add(Conv2D(64, (3, 3), activation='relu'))
facial_model.add(Conv2D(64, (3, 3), activation='relu'))
facial_model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

facial_model.add(Conv2D(128, (3, 3), activation='relu'))
facial_model.add(Conv2D(128, (3, 3), activation='relu'))
facial_model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

facial_model.add(Flatten())

facial_model.add(Dense(1024, activation='relu'))
facial_model.add(Dropout(0.3))
facial_model.add(Dense(1024, activation='relu'))
facial_model.add(Dropout(0.3))

#output of this layer corresponds to number of classes total emotion to be detected 
facial_model.add(Dense(numClasses, activation='softmax'))

In [ ]:
#gives total parameters and output shape so that it is easy to understand
facial_model.summary()

In [ ]:
#processing in batch fot random input so that there will be no bias due to less data
gen = ImageDataGenerator()
trainGenerator = gen.flow(x_train, y_train, batch_size=batchSize)

In [ ]:
facial_model.compile(loss='categorical_crossentropy'
    , optimizer=keras.optimizers.Adam()
    , metrics=['accuracy']
)

In [ ]:
facial_model.fit_generator(trainGenerator, steps_per_epoch=batchSize, epochs=epochs)

In [ ]:
train_score = facial_model.evaluate(x_train, y_train, verbose=0)
print('Train loss:', train_score[0])
print('Train accuracy:', 100*train_score[1])
 
test_score = facial_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', test_score[0])
print('Test accuracy:', 100*test_score[1])

In [ ]:
#for confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
 
pred_list = []; actual_list = []
 
for i in facial_model.predict(x_test):
  pred_list.append(np.argmax(i))

for i in y_test:
  actual_list.append(np.argmax(i))

confusion_matrix(actual_list, pred_list)

In [ ]:
#To draw barchart
def analyse(emo_object):
    emotion_classes = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    y_pos = np.arange(len(emotion_classes))
    
    plt.bar(y_pos, emo_object, align='center', alpha=0.5)
    plt.xticks(y_pos, emotion_classes)
    plt.ylabel('percent')
    plt.title('result ofemotion')
    plt.show()

In [ ]:
#take image and put it in root folder and below code test it for emotion.
img = image.load_img("./a.jpg", grayscale=True, target_size=(48, 48))

x_new = image.img_to_array(img)
x_new = np.expand_dims(x, axis = 0)

x_new /= 255

custom = facial_model.predict(x_new)
analyse(custom[0])

x_new = np.array(x_new, 'float32')
x_new = x_new.reshape([48, 48]);

plt.gray()
plt.imshow(x_new)
plt.show()